## Tutorial on how to 'delay' the start of particle advection

In many applications, it is needed to 'delay' the start of particle advection. For example because particles need to be released at different times throughout an experiment. Or because particles need to be released at a conatant rate from the same set of locations.

This tutorial will show how this can be done. We start with importing the relevant modules.

In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, plotTrajectoriesFile
from parcels import AdvectionRK4
import numpy as np
from datetime import timedelta as delta

First import a `FieldSet` (from the Peninsula example, in this case)

In [2]:
fieldset = FieldSet.from_parcels('Peninsula_data/peninsula', allow_time_extrapolation = True)

Now, there are two ways to delay the start of particles. Either by defining the whole `ParticleSet` at initialisation and giving each particle its own `time`. Or by using the `repeatdt` argument. We will show both options here

### Assigning each particle its own `time` ###

The simplest way to delaye the start of a particle is to use the `time` argument for each particle

In [3]:
npart = 10  # number of particles to be released
lon = 0.025 * np.ones(npart)
lat = np.linspace(0.025 , 0.4, npart, dtype=np.float32)
time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later

pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lon, lat=lat, time=time)

Then we can execute the `pset` as usual

In [4]:
output_file = pset.ParticleFile(name="DelayParticle_time", outputdt=delta(hours=1))
pset.execute(AdvectionRK4, runtime=delta(hours=24), dt=delta(minutes=5),
             output_file=output_file)

INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gn/T/parcels-501/4b31da299158ef673e01491b786d6e1b.so


And then finally, we can show a movie of the particles. Note that the southern-most particles start to move first.

In [5]:
plotTrajectoriesFile('DelayParticle_time.nc', mode='movie2d_notebook')

### Using the `repeatdt` argument ###

The second method to delay the start of particle releases is to use the `repeatdt` argument when constructing a `ParticleSet`. This is especially useful if you want to repeatedly release particles from the same set of locations.

In [6]:
npart = 10  # number of particles to be released
lon = 0.025 * np.ones(npart)
lat = np.linspace(0.025 , 0.4, npart, dtype=np.float32)
repeatdt = delta(hours=3)  # release from the same set of locations every 3 hours

pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lon, lat=lat, repeatdt=repeatdt)

Now we again define an output file and execute the `pset` as usual.

In [7]:
output_file = pset.ParticleFile(name="DelayParticle_releasedt", outputdt=delta(hours=1))
pset.execute(AdvectionRK4, runtime=delta(hours=24), dt=delta(minutes=5),
             output_file=output_file)

INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gn/T/parcels-501/bcc7e3a29987978e9d717aaf17db0773.so


In [9]:
plotTrajectoriesFile('DelayParticle_releasedt.nc', mode='movie2d_notebook')

And we get an animation where a new particle is released every 3 hours from each start location

In [10]:
convert_IndexedOutputToArray('DelayParticle_releasedt.nc', 'DelayParticle_releasedt_array.nc')

100% (90 of 90) |###################| Elapsed Time: 0:00:00 Time: 0:00:00


And we can also make a movie of this file. Note that generating movies for Array-style particle files tends to be much faster than for Indexed-style files, particularly when the files become very big.

In [11]:
plotTrajectoriesFile('DelayParticle_releasedt.nc', mode='movie2d_notebook')